In [1]:
!pip install -q -U datasets neptune neptune-pytorch torchtext transformers bitsandbytes huggingface_hub[cli]

zsh:1: no matches found: huggingface_hub[cli]


In [2]:
import os

os.environ['NEPTUNE_PROJECT'] = 'ling583/ling583-anish'
os.environ[
    'NEPTUNE_API_TOKEN'] = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxNzU4ZjdlZi1lZjhkLTQxZmItOGYzOC03ZThkMDVlMDVmZWUifQ=='

In [3]:
import pandas as pd
import neptune
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from datasets import Dataset, DatasetDict

pd.options.mode.copy_on_write = True
pd.options.display.max_colwidth = 500

/Users/anishchintamaneni/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_json("hf://datasets/jonathanli/legal-advice-reddit/test.jsonl", lines=True)
df["label"] = df["text_label"]
df["text"] = df["title"] + " " + df["body"]

df, train_df = train_test_split(
    df, test_size=500, stratify=df["text_label"], random_state=1
)
df, valid_df = train_test_split(
    df, test_size=1000, stratify=df["text_label"], random_state=1
)
df, test_df = train_test_split(
    df, test_size=1000, stratify=df["text_label"], random_state=1
)
df, minitest_df = train_test_split(
    df, test_size=250, stratify=df["text_label"], random_state=1
)

len(train_df), len(valid_df), len(test_df), len(minitest_df)


(500, 1000, 1000, 250)

In [5]:
dataset = DatasetDict()
dataset["train"] = Dataset.from_pandas(train_df)
dataset["valid"] = Dataset.from_pandas(valid_df)
dataset["test"] = Dataset.from_pandas(test_df)
dataset["minitest"] = Dataset.from_pandas(minitest_df)

dataset = dataset.class_encode_column("label")

dataset.save_to_disk("lab08_dataset")

Casting to class labels: 100%|██████| 250/250 [00:00<00:00, 92695.90 examples/s]
Saving the dataset (1/1 shards): 100%|█| 500/500 [00:00<00:00, 173476.05 example
Saving the dataset (1/1 shards): 100%|█| 1000/1000 [00:00<00:00, 284263.23 examp
Saving the dataset (1/1 shards): 100%|█| 1000/1000 [00:00<00:00, 312238.81 examp
Saving the dataset (1/1 shards): 100%|█| 250/250 [00:00<00:00, 74031.06 examples


In [6]:
svm = make_pipeline(TfidfVectorizer(lowercase=True), LinearSVC(C=1, dual="auto"))
svm.fit(dataset["train"]["text_label"], dataset["train"]["text_label"])
predicted = svm.predict(dataset["test"]["text"])
print(classification_report(dataset["test"]["text_label"], predicted, zero_division=0))

              precision    recall  f1-score   support

    business       0.48      0.55      0.52        58
    contract       0.44      0.62      0.51        52
    criminal       0.79      0.12      0.22        88
     digital       0.33      0.02      0.04        53
     driving       0.77      0.28      0.42        95
  employment       0.85      0.14      0.24       161
      family       0.31      0.22      0.26       121
     housing       0.32      0.71      0.44       241
   insurance       0.50      0.91      0.65        44
      school       0.43      0.80      0.56        44
       wills       0.00      0.00      0.00        43

    accuracy                           0.40      1000
   macro avg       0.47      0.40      0.35      1000
weighted avg       0.50      0.40      0.35      1000



In [7]:
mini_predicted = svm.predict(dataset["minitest"]["text"])
print(
    classification_report(
        dataset["minitest"]["text_label"], mini_predicted, zero_division=0
    )
)

              precision    recall  f1-score   support

    business       0.50      0.40      0.44        15
    contract       0.33      0.46      0.39        13
    criminal       0.40      0.09      0.15        22
     digital       0.00      0.00      0.00        13
     driving       0.50      0.08      0.14        24
  employment       0.67      0.05      0.09        40
      family       0.32      0.27      0.29        30
     housing       0.27      0.65      0.39        60
   insurance       0.44      1.00      0.61        11
      school       0.47      0.64      0.54        11
       wills       1.00      0.09      0.17        11

    accuracy                           0.34       250
   macro avg       0.45      0.34      0.29       250
weighted avg       0.43      0.34      0.27       250



In [8]:
run = neptune.init_run(
    name="LinearSVC",
    description="TfidfVectorizer()+LinearSVC() with C=1",
    tags=["lab-08"],
)

run["dataset"].track_files("lab08_dataset")

run["score/accuracy"] = accuracy_score(dataset["test"]["text_label"], predicted)
run["score/f1_macro"] = f1_score(
    dataset["test"]["text_label"], predicted, average="macro"
)
run["score/mini_accuracy"] = accuracy_score(
    dataset["minitest"]["text_label"], mini_predicted
)
run["score/mini_f1_macro"] = f1_score(
    dataset["minitest"]["text_label"], mini_predicted, average="macro"
)


run.stop()

[neptune] [warning] NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/ling583/ling583-anish/e/LING-46
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 5 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 5 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/ling583/ling583-anish/e/LING-46/metadata
